In [13]:
import os
import glob
import SimpleITK as sitk
import numpy as np
import pandas as pd


# 设置路径
data_dir = './videos/bed_rest_MRI/bed_rest_MRI_L4L5'

label_dir = os.path.join(data_dir, 'label_in_png_L4L5_4class_DINO_order')
prompt_dir = 'manual_prompt_frame0_1_2'
prompt_dir = 'manual_prompt_frame0_1'
prompt_dir = 'manual_prompt_frame0'

auto_seg_dir = os.path.join(data_dir, prompt_dir, 'SAM2_seg_mask_nolap')


# 创建新的目标目录
output_dir = os.path.join(data_dir, prompt_dir, 'seg_mask_renamed')
os.makedirs(output_dir, exist_ok=True)

In [14]:


# 目录设置
auto_seg_dir = os.path.join(data_dir, prompt_dir, 'seg_in_nii_72_L4-L5')    # 自动分割结果所在目录
manual_seg_dir = os.path.join(data_dir, 'label_in_nii_72_L4-L5')             # 手动分割结果所在目录

# 创建 DataFrame 用于存储结果
df_results = pd.DataFrame(columns=["PID", "Class", "Auto Area", "Manual Area"])

# 遍历自动分割结果文件
for idx, auto_filename in enumerate(os.listdir(auto_seg_dir)):
    print(f"{idx}: Processing {auto_filename}")

    # 提取 PID，假设文件名格式类似 "ID1A_R0_XXX.nii.gz"，PID 为前两段 "ID1A_R0"
    parts = auto_filename.split('_')
    pid = "_".join(parts[0:3])

    # 自动分割文件读取
    auto_path = os.path.join(auto_seg_dir, auto_filename)
    auto_img = sitk.ReadImage(auto_path)
    auto_array = sitk.GetArrayFromImage(auto_img)
    auto_spacing = auto_img.GetSpacing()  # 例如 (spacing_x, spacing_y, spacing_z)

    # 查找对应的手动分割文件（根据 PID 匹配）
    manual_files = glob.glob(os.path.join(manual_seg_dir, pid + "*.nii.gz"))
    if not manual_files:
        print(f"Warning: Manual segmentation file not found for {pid}")
        continue
    manual_path = manual_files[0]
    manual_img = sitk.ReadImage(manual_path)
    manual_array = sitk.GetArrayFromImage(manual_img)
    manual_spacing = manual_img.GetSpacing()

    # 对于每个类别（例如 1,2,3,4）计算面积（单位：原面积除以 100，并保留 3 位小数）
    for class_value in [1, 2, 3, 4]:
        # 自动分割中该类别的像素数及面积
        auto_mask = (auto_array == class_value)
        auto_pixel_count = np.sum(auto_mask)
        auto_area = auto_pixel_count * auto_spacing[0] * auto_spacing[1]
        auto_area = round(auto_area / 100, 3)

        # 手动分割中该类别的像素数及面积
        manual_mask = (manual_array == class_value)
        manual_pixel_count = np.sum(manual_mask)
        manual_area = manual_pixel_count * manual_spacing[0] * manual_spacing[1]
        manual_area = round(manual_area / 100, 3)

        # 将结果追加到 DataFrame 中
        df_results = df_results._append({
            "PID": pid,
            "Class": class_value,
            "Auto Area": auto_area,
            "Manual Area": manual_area
        }, ignore_index=True)

# 根据 prompt_dir 过滤掉指定前缀的 PID
manual_prompt_id = ['ID4D_BDC', 'ID12M_R13', 'ID15Q_R13']
# 逻辑：根据 prompt_dir 中是否包含 "1" 或 "2" 来决定去除 manual_prompt_id 中的前几项
remove_prefixes = manual_prompt_id[: 1 + ("1" in prompt_dir) + ("2" in prompt_dir)]
print("Remove prefixes:", remove_prefixes)
filter_pid = lambda df: df[~df["PID"].astype(str).str.startswith(tuple(remove_prefixes))]
df_results = filter_pid(df_results)



0: Processing ID9J_R0_L4-L5_sam2_seg_slice23_seg.nii.gz
1: Processing ID24Z_BDC_L4-L5_sam2_seg_slice15_seg.nii.gz
2: Processing ID10K_BDC_L4-L5_sam2_seg_slice42_seg.nii.gz
3: Processing ID12M_BDC_L4-L5_sam2_seg_slice13_seg.nii.gz
4: Processing ID22X_R13_L4-L5_sam2_seg_slice22_seg.nii.gz
5: Processing ID17S_BDC_L4-L5_sam2_seg_slice20_seg.nii.gz
6: Processing ID7G_R13_L4-L5_sam2_seg_slice12_seg.nii.gz
7: Processing ID15Q_R0_L4-L5_sam2_seg_slice22_seg.nii.gz
8: Processing ID8H_R0_L4-L5_sam2_seg_slice24_seg.nii.gz
9: Processing ID14P_R0_L4-L5_sam2_seg_slice21_seg.nii.gz
10: Processing ID14P_R13_L4-L5_sam2_seg_slice24_seg.nii.gz
11: Processing ID6F_BDC_L4-L5_sam2_seg_slice17_seg.nii.gz
12: Processing ID19U_R0_L4-L5_sam2_seg_slice22_seg.nii.gz
13: Processing ID13N_R0_L4-L5_sam2_seg_slice35_seg.nii.gz
14: Processing ID23Y_R0_L4-L5_sam2_seg_slice11_seg.nii.gz
15: Processing ID1A_R0_L4-L5_sam2_seg_slice17_seg.nii.gz
16: Processing ID5E_R0_L4-L5_sam2_seg_slice19_seg.nii.gz
17: Processing ID20V_R

In [15]:
print(df_results.shape)
df_results.head()


(284, 4)


PID Class  Auto Area  Manual Area
0    ID9J_R0_L4-L5     1     14.639       14.421
1    ID9J_R0_L4-L5     2     10.667       11.329
2    ID9J_R0_L4-L5     3     10.396       11.014
3    ID9J_R0_L4-L5     4     15.897       14.443
4  ID24Z_BDC_L4-L5     1     16.396        7.053

In [16]:
import numpy as np
import pandas as pd
from scipy.stats import t
import statsmodels.formula.api as smf
import pingouin as pg
import warnings

warnings.filterwarnings('ignore')

# 先拆分 Person_ID 和 Timepoint
df_results['Person_ID'] = df_results['PID'].str.split('_').str[0]
df_results['Timepoint'] = df_results['PID'].str.split('_').str[1]

# 类别标签映射
class_names = {
    1: "Right_LES",
    2: "Right_MF",
    3: "Left_MF",
    4: "Left_LES"
}
print_order = [4, 1, 3, 2]

alpha = 0.05
fixed_margin_pct = 0.05  # 5% margin for TOST
min_patients_for_lmm = 3
min_observations_for_class = 10

results = []

for class_value in print_order:
    region_name = class_names[class_value]
    df_class = df_results[df_results["Class"] == class_value]
    if df_class.empty:
        continue

    # 计算基本统计值
    manual_vals = df_class["Manual Area"].values
    auto_vals = df_class["Auto Area"].values
    manual_mean, manual_sd = np.mean(manual_vals), np.std(manual_vals, ddof=1)
    auto_mean, auto_sd = np.mean(auto_vals), np.std(auto_vals, ddof=1)
    manual_str = f"{manual_mean:.2f}±{manual_sd:.2f}"
    ai_str = f"{auto_mean:.2f}±{auto_sd:.2f}"

    # 计算 MAE & MAPE
    abs_diff = np.abs(auto_vals - manual_vals)
    mae = np.mean(abs_diff)
    with np.errstate(divide='ignore', invalid='ignore'):
        pct_err = np.where(manual_vals != 0, abs_diff / np.abs(manual_vals), np.nan)
    mape = np.nanmean(pct_err) * 100 if not np.all(np.isnan(pct_err)) else np.nan
    mae_str = f"{mae:.2f}"
    mape_str = f"{mape:.1f}%" if not np.isnan(mape) else "NA"

    # 构造长格式用于混合模型与 ICC
    n = len(df_class)
    long_df = pd.DataFrame({
        "Person_ID": list(df_class["Person_ID"]) * 2,
        "Timepoint": list(df_class["Timepoint"]) * 2,
        "volume": np.concatenate([auto_vals, manual_vals]),
        "Method": ["Auto"] * n + ["Manual"] * n
    })
    long_df['ai_derived'] = (long_df['Method'] == 'Auto').astype(int).astype('category')
    long_df['Timepoint'] = long_df['Timepoint'].astype('category')

    # 样本量检查
    n_patients = long_df['Person_ID'].nunique()
    n_obs = len(long_df)
    if n_patients < min_patients_for_lmm or n_obs < min_observations_for_class:
        warnings.warn(f"{region_name} 样本量不足，跳过")
        continue

    # 混合效应模型: volume ~ C(ai_derived) + C(Timepoint) + (1|Person_ID)
    md = smf.mixedlm(
        "volume ~ C(ai_derived) + C(Timepoint)",
        data=long_df,
        groups="Person_ID",
        re_formula="1"
    )
    res = md.fit(reml=True, method='powell', maxiter=5000)

    beta = res.params.get("C(ai_derived)[T.1]", np.nan)
    se = res.bse.get("C(ai_derived)[T.1]", np.nan)
    df_resid = res.df_resid

    # 固定 ±5% TOST 计算
    fixed_margin = fixed_margin_pct * manual_mean
    t_lower = (beta + fixed_margin) / se
    p_lower = 1 - t.cdf(t_lower, df_resid)
    t_upper = (beta - fixed_margin) / se
    p_upper = t.cdf(t_upper, df_resid)
    tost_p_fixed = max(p_lower, p_upper)

    # 最小等效 margin 计算
    tcrit_one = t.ppf(1 - alpha, df_resid)
    min_margin_abs = abs(beta) + se * tcrit_one
    if manual_mean != 0:
        min_margin_pct = min_margin_abs / abs(manual_mean) * 100
        equiv_margin_str = f"{min_margin_pct:.2f}%"
    else:
        equiv_margin_str = "NA"

    # 计算 TOST p 在最小margin时 (基本接近 alpha)
    t_lower_min = (beta + min_margin_abs) / se
    p_lower_min = 1 - t.cdf(t_lower_min, df_resid)
    t_upper_min = (beta - min_margin_abs) / se
    p_upper_min = t.cdf(t_upper_min, df_resid)
    tost_p_min = max(p_lower_min, p_upper_min)

    # ICC 计算
    icc_val, icc_l, icc_u = np.nan, np.nan, np.nan
    try:
        icc_df = pg.intraclass_corr(
            data=long_df,
            targets="Person_ID",
            raters="Method",
            ratings="volume"
        )
        icc2 = icc_df[icc_df["Type"] == "ICC2"].iloc[0]
        icc_val, icc_l, icc_u = icc2["ICC"], icc2["CI95%"][0], icc2["CI95%"][1]
    except Exception:
        warnings.warn(f"{region_name} ICC计算失败")

    icc_str = f"{icc_val:.3f}" if not np.isnan(icc_val) else "NA"
    ci_str = f"[{icc_l:.2f}, {icc_u:.2f}]" if not np.isnan(icc_l) else "NA"

    results.append({
        'Muscle Region': region_name,
        'Manual (M±SD)': manual_str,
        'AI (M±SD)': ai_str,
        'MAE': mae_str,
        'MAPE': mape_str,
        'TOST p (±5%)': tost_p_fixed,
        'Equiv Margin (%)': equiv_margin_str,
        'TOST p (min margin)': tost_p_min,
        'ICC': icc_str,
        '95% CI': ci_str
    })

df_out = pd.DataFrame(results, columns=[
    'Muscle Region',
    'Manual (M±SD)',
    'AI (M±SD)',
    'MAE',
    'MAPE',
    'TOST p (±5%)',
    'Equiv Margin (%)',
    'TOST p (min margin)',
    'ICC',
    '95% CI'
])


In [17]:
df_out # 1shot

Muscle Region Manual (M±SD)   AI (M±SD)   MAE   MAPE  TOST p (±5%)  \
0      Left_LES    12.72±2.99  11.85±4.87  2.48  19.0%      0.704057   
1     Right_LES    12.58±3.23  15.23±4.70  3.46  30.5%      0.999998   
2       Left_MF     9.02±1.90  10.23±3.16  1.78  21.6%      0.993609   
3      Right_MF     9.39±2.16  10.08±4.28  2.04  23.0%      0.731574   

  Equiv Margin (%)  TOST p (min margin)    ICC         95% CI  
0           12.42%                 0.05  0.614   [0.30, 0.81]  
1           26.66%                 0.05  0.592   [0.00, 0.84]  
2           19.05%                 0.05  0.327  [-0.04, 0.63]  
3           13.85%                 0.05  0.629   [0.32, 0.82]

In [ ]:
ss

In [12]:
df_out # 2shot

Muscle Region Manual (M±SD)   AI (M±SD)   MAE   MAPE  TOST p (±5%)  \
0      Left_LES    12.73±3.02  12.71±4.28  1.70  12.7%      0.065559   
1     Right_LES    12.56±3.25  13.20±4.22  1.74  14.4%      0.510935   
2       Left_MF     9.01±1.91   9.96±2.11  1.21  14.5%      0.999455   
3      Right_MF     9.39±2.17   9.68±2.94  1.20  14.3%      0.187461   

  Equiv Margin (%)  TOST p (min margin)    ICC        95% CI  
0            5.44%                 0.05  0.812  [0.61, 0.91]  
1            9.74%                 0.05  0.822  [0.63, 0.92]  
2           13.17%                 0.05  0.772  [0.22, 0.92]  
3            6.70%                 0.05  0.815  [0.62, 0.92]

In [7]:
df_out # 3shot

Muscle Region Manual (M±SD)   AI (M±SD)   MAE   MAPE  TOST p (±5%)  \
0      Left_LES    12.79±2.99  12.29±3.93  1.68  12.6%      0.357656   
1     Right_LES    12.60±3.25  12.84±3.76  1.53  12.4%      0.106222   
2       Left_MF     8.99±1.92   9.66±2.08  0.99  11.7%      0.936521   
3      Right_MF     9.36±2.17  10.18±2.83  1.19  14.0%      0.944936   

  Equiv Margin (%)  TOST p (min margin)    ICC        95% CI  
0            9.03%                 0.05  0.833  [0.65, 0.92]  
1            6.02%                 0.05  0.854  [0.69, 0.93]  
2           10.11%                 0.05  0.852  [0.49, 0.95]  
3           12.56%                 0.05  0.769  [0.48, 0.90]